# Wind Turbine Power Prediction

In this study I am going to predict a wind turbine power production by using the wind speed, wind direction, month and hour data.

The dataset consists of 50530 observations. In order to demonstrate my data science skills with big data, I am going to use Pyspark library.

The dataset contains:

* Date/Time (for 10 minutes intervals)
* LV ActivePower (kW): The power generated by the turbine for that moment
* Wind Speed (m/s): The wind speed at the hub height of the turbine (the wind speed that turbine use for electricity generation)
* TheoreticalPowerCurve (KWh): The theoretical power values that the turbine generates with that wind speed which is given by the turbine manufacturer
* Wind Direction (°): The wind direction at the hub height of the turbine (wind turbines turn to this direction automaticly)

Dataset Resource: 

https://www.kaggle.com/berkerisen/wind-turbine-scada-dataset


## Aim of the Study:

**My aim is to predict wind turbine power production from the wind speed, wind direction, month of the year and the hour of the day.** 

## Importing Libraries and Spark Configuration

In [ ]:
import os
import sys

# Check for required environment variables, and set if not present
java_home = os.getenv("JAVA_HOME", "/usr/lib/jvm/default-java")  # General placeholder for Unix-based systems
hadoop_home = os.getenv("HADOOP_HOME", "/usr/local/hadoop")  # Default path for Hadoop on many Unix-based systems

# Set environment variables
os.environ["JAVA_HOME"] = java_home
os.environ["HADOOP_HOME"] = hadoop_home
os.environ["HADOOP.HOME.DIR"] = hadoop_home

from pathlib import Path

os.environ["PATH"] += f";{Path(hadoop_home) / 'bin'}"

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

sns.set_style("white")

from warnings import filterwarnings

filterwarnings("ignore")

# Importing pyspark libraries
from pyspark.sql import SparkSession

# Allow the user to set the Python path via environment variable or use the default system Python
python_path = os.getenv("PYSPARK_PYTHON", os.path.realpath(sys.executable))

os.environ["PYSPARK_PYTHON"] = python_path
os.environ["PYSPARK_DRIVER_PYTHON"] = python_path

# Create the Spark session with dynamic configurations
spark = (
    SparkSession.builder.master("local[*]")
    .appName("wind_turbine_project")
    .config("spark.executor.memory", "4g")
    .config("spark.pyspark.python", python_path)
    .config("spark.pyspark.driver.python", python_path)
    .getOrCreate()
)

sc = spark.sparkContext

sc

## Reading the Dataset

In [ ]:
# Reading the dataset as Spark DataFrame
spark_df = spark.read.csv("T1.csv", header=True, inferSchema=True)

# Caching the dataset
spark_df.cache()

# Converting all the column names to lower case
spark_df = spark_df.toDF(*[c.lower() for c in spark_df.columns])

print("Show the first 5 rows")
print(spark_df.show(5))
print()
print("What are the variable data types?")
print(spark_df.printSchema())
print()
print("How many observations do we have?")
print(spark_df.count())

In [ ]:
# Extracting a substring from columns to create month and hour variables

from pyspark.sql.functions import substring

spark_df = spark_df.withColumn("month", substring("date/time", 4, 2))
spark_df = spark_df.withColumn("hour", substring("date/time", 12, 2))

# Converting string month and hour variables to integer
from pyspark.sql.types import IntegerType

spark_df = spark_df.withColumn("month", spark_df.month.cast(IntegerType()))
spark_df = spark_df.withColumn("hour", spark_df.hour.cast(IntegerType()))

print(spark_df.show(5))

## Exploratory Data Analysis

In [ ]:
pd.options.display.float_format = "{:.2f}".format
spark_df.select("wind speed (m/s)", "theoretical_power_curve (kwh)", "lv activepower (kw)").toPandas().describe()

### Question: What are the distributions of the variables?

**For creating visualization we need to either use aggregated data or use a sample from the big data.**

**So I will get a random sample from my big data.**

In [ ]:
# Taking a random sample from the big data
sample_df = spark_df.sample(withReplacement=False, fraction=0.1, seed=42).toPandas()

# Visualizing the distributions with the sample data
columns = [
    "wind speed (m/s)",
    "wind direction (°)",
    "month",
    "hour",
    "theoretical_power_curve (kwh)",
    "lv activepower (kw)",
]
i = 1
plt.figure(figsize=(10, 12))
for each in columns:
    plt.subplot(3, 2, i)
    sample_df[each].plot.hist(bins=12)
    plt.title(each)
    i += 1

### Question: Is there any difference between the months for average power production ?

In [ ]:
# Average power production by month
monthly = spark_df.groupby("month").mean("lv activepower (kw)").sort("avg(lv activepower (kw))").toPandas()
sns.barplot(x="month", y="avg(lv activepower (kw))", data=monthly)
plt.title("Months and Average Power Production");

### Question: Is there any difference between the hours for average power production?

In [ ]:
# Average power production by hour
hourly = spark_df.groupby("hour").mean("lv activepower (kw)").sort("avg(lv activepower (kw))").toPandas()
sns.barplot(x="hour", y="avg(lv activepower (kw))", data=hourly)
plt.title("Hours and Average Power Production");

### Question: Is there any correlation between the wind speed, wind direction and power production?

In [ ]:
display(sample_df[columns].corr())
sns.pairplot(sample_df[columns], markers="*");

**Wind speed and power production is highly correlated as one would expect.**

**We can see there are lower level power production for some wind directions.**

### Question: What is the average power production level for different wind speeds?

In [ ]:
# Finding average power production for 5 m/s wind speed increments
wind_speed = []
avg_power = []
for i in [0, 5, 10, 15, 20]:
    avg_value = (
        spark_df.filter((spark_df["wind speed (m/s)"] > i) & (spark_df["wind speed (m/s)"] <= i + 5))
        .agg({"lv activepower (kw)": "mean"})
        .collect()[0][0]
    )
    avg_power.append(avg_value)
    wind_speed.append(str(i) + "-" + str(i + 5))

sns.barplot(x=wind_speed, y=avg_power, color="orange")
plt.title("Avg Power Production for 5 m/s Wind Speed Increments")
plt.xlabel("Wind Speed")
plt.ylabel("Average Power Production");

**From the graph above we can see the power production reaches near a maximum level after the wind speed reaches 15 m/s.**

### Question: What is the power production for different wind directions and speeds? 

**Let's create a polar diagram with wind speed, wind direction and power production from the sample data.**

In [ ]:
# Creating the polar diagram
from math import radians

plt.figure(figsize=(8, 8))
ax = plt.subplot(111, polar=True)
# Inside circles are the wind speed and marker color and size represents the amount of power production
sns.scatterplot(
    x=[radians(x) for x in sample_df["wind direction (°)"]],
    y=sample_df["wind speed (m/s)"],
    size=sample_df["lv activepower (kw)"],
    hue=sample_df["lv activepower (kw)"],
    alpha=0.7,
    legend=None,
)
# Setting the polar diagram's top represents the North
ax.set_theta_zero_location("N")
# Setting -1 to start the wind direction clockwise
ax.set_theta_direction(-1)
# Setting wind speed labels in a better position to see
ax.set_rlabel_position(110)
plt.title("Wind Speed - Wind Direction - Power Production Diagram")
plt.ylabel(None);

**We can see that the wind turbine produces more power if the wind blows from the directions between 000-090 and 180-225 degrees.**

### Question: Does the manufacturer's theoritical power production curve fit well with the real production?

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(
    x="wind speed (m/s)", y="lv activepower (kw)", color="orange", label="Real Production", alpha=0.5, data=sample_df
)
sns.lineplot(
    x="wind speed (m/s)",
    y="theoretical_power_curve (kwh)",
    color="blue",
    label="Theoritical Production",
    data=sample_df,
)
plt.title("Wind Speed and Power Production Chart")
plt.ylabel("Power Production (kw)");

**From the graph above, we can see the theoritical power production curve generally fits well with the real production.**

**We can see the power production reaches a maximum level and continues in a straight line if the wind speed reaches to 15 m/s.**

**Also we can see there are some 0 power production, even the wind speed is higher than 5 m/s. I want to investigate the reason.**

**But before what is the minimum wind speed for theoritical power production curve?**

### Question: What is the wind speed threshold value for zero theorical power?

In [ ]:
# Filter the big data where the real and theoritical power productions are equal to 0
zero_theo_power = spark_df.filter(
    (spark_df["lv activepower (kw)"] == 0) & (spark_df["theoretical_power_curve (kwh)"] == 0)
).toPandas()

display(zero_theo_power[["wind speed (m/s)", "theoretical_power_curve (kwh)", "lv activepower (kw)"]].sample(5))

# Let's see the wind speed distribution for 0 power production
zero_theo_power["wind speed (m/s)"].hist()
plt.title("Wind Speed Distribution for 0 Power Production")
plt.xlabel("Wind speed (m/s)")
plt.ylabel("Counts for 0 Power Production");

**We can see from above, limit for the theoritical power curve is 3 m/s wind speed. If the wind speed is below 3 m/s model doesn't expect any power production.**

**But there are some observations for 0 power production even the wind speed is more than 3 m/s.**

### Question: Why there aren't any power production in some observations while the wind speed is higher than 3 m/s?

In [ ]:
# Observations for the wind speed > 3m/s and power production = 0,
# While theoritically there should be power production
zero_power = spark_df.filter(
    (spark_df["lv activepower (kw)"] == 0)
    & (spark_df["theoretical_power_curve (kwh)"] != 0)
    & (spark_df["wind speed (m/s)"] > 3)
).toPandas()
display(zero_power.head())
print("No of Observations (while Wind Speed > 3 m/s and Power Production = 0): ", len(zero_power))

**There are 3497 observations where theoritically there should be power production. From the dataset we cannot see the reason, it might be caused by maintenance. But let's see if we can see any information from the wind speed, direction and month?**

In [ ]:
zero_power["wind speed (m/s)"].plot.hist(bins=8)
plt.xlabel("Wind Speed (m/s)")
plt.ylabel("Counts for Zero Production")
plt.title("Wind Speed Counts for Zero Power Production")
plt.xticks(ticks=np.arange(4, 18, 2));

**It looks like theoritically wind speed threshold should be 4 m/s. But there are also other observations with zero power production while the wind speed is higher.**

**Let's see the monthly distribution for zero power production.**

In [ ]:
sns.countplot(zero_power, x="month");

**It is usually in December and January when the wind turbine doesn't produce production.**

**Because I cannot decide if these zero power productions are caused by maintenance periods or something else, I am going to accept those 3497 observations as outliers and remove them from the dataset.**

In [ ]:
# Excluding the observations meeting the filter criterias
spark_df = spark_df.filter(
    ~(
        (spark_df["lv activepower (kw)"] == 0)
        & (spark_df["theoretical_power_curve (kwh)"] != 0)
        & (spark_df["wind speed (m/s)"] > 3)
    )
)

### Question: Is there any other outliers?

In [ ]:
columns = ["wind speed (m/s)", "wind direction (°)", "theoretical_power_curve (kwh)", "lv activepower (kw)"]
i = 1
plt.figure(figsize=(20, 3))
for each in columns:
    pandas_df = spark_df.select(each).toPandas()
    plt.subplot(1, 4, i)
    sns.boxplot(pandas_df)
    plt.title(each)
    i += 1

**From the graphs above I can see there are some outliers in the wind speed data.**

**I will find the upper and lower threshold values for the wind speed data, and I will analyze the outliers.**

In [ ]:
# Create a pandas df for visualization
wind_speed = spark_df.select("wind speed (m/s)").toPandas()

# Defining the quantiles and interquantile range
Q1 = wind_speed["wind speed (m/s)"].quantile(0.25)
Q3 = wind_speed["wind speed (m/s)"].quantile(0.75)
IQR = Q3 - Q1
# Defining the lower and upper threshold values
lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR

print("Quantile (0.25): ", Q1, "  Quantile (0.75): ", Q3)
print("Lower threshold: ", lower, " Upper threshold: ", upper)

In [ ]:
# Fancy indexing for outliers
outlier_tf = (wind_speed["wind speed (m/s)"] < lower) | (wind_speed["wind speed (m/s)"] > upper)

print("Total Number of Outliers: ", len(wind_speed["wind speed (m/s)"][outlier_tf]))
print("--" * 15)
print("Some Examples of Outliers:")
print(wind_speed["wind speed (m/s)"][outlier_tf].sample(10))

**It is a rare event for wind speed to be over 19 m/s in our dataset.**

**Out of 47033, there is only 407 observations while the wind speed is over 19 m/s.**

**Now I want to see average power production for these high wind speed.**

In [ ]:
spark_df.select("wind speed (m/s)", "lv activepower (kw)").filter(spark_df["wind speed (m/s)"] >= 19).agg(
    {"lv activepower (kw)": "mean"}
).show()

**So instead of erasing the outliers, I am going to set the wind speed as 19 m/s for those observations.**

In [ ]:
from pyspark.sql import functions as f

spark_df = spark_df.withColumn(
    "wind speed (m/s)", f.when(f.col("wind speed (m/s)") > 19.447, 19).otherwise(f.col("wind speed (m/s)"))
)
spark_df.count()

### Question: What are the general criterias for power production?

**It is important to understand the pattern in the data. We should learn the data before the machine.**

**1. We saw from the graph that in March, August and November, the average power production is higher.**

**2. The average power production is higher daily between 16:00 and 24:00.**

**3. The power production is higher when the wind blows from the directions between 000-090 and 180-225 degrees.**

**So let's try to predict a high and low level of power production from the criterias above before ML algorithm.**

In [ ]:
# High level power production
spark_df.filter(
    ((spark_df["month"] == 3) | (spark_df["month"] == 8) | (spark_df["month"] == 11))
    & ((spark_df["hour"] >= 16) | (spark_df["hour"] <= 24))
    & ((spark_df["wind direction (°)"] > 0) | (spark_df["wind direction (°)"] < 90))
    & ((spark_df["wind direction (°)"] > 180) | (spark_df["wind direction (°)"] < 225))
).agg({"lv activepower (kw)": "mean"}).show()

In [ ]:
# Low level power production
spark_df.filter(
    (spark_df["month"] == 7)
    & ((spark_df["hour"] >= 9) | (spark_df["hour"] <= 11))
    & ((spark_df["wind direction (°)"] > 90) | (spark_df["wind direction (°)"] < 160))
).agg({"lv activepower (kw)": "mean"}).show()

## Data Preparation for ML Algorithms

**After analysing and understanding the dataset, we can build a ML regression model to predict wind turbine power production by using the wind speed, wind direction, month of the year and hour of the day.**

**Using ML algorithms with Spark is a bit different from well known Sckitlearn library.**

**We need to feed the model with a dataframe made of variables compressed in vectors called as 'features', and target variable as 'label'. For these convertions I am going to use VectorAssembler method from Pyspark.**

In [ ]:
# Preparing the independent variables (Features)
from pyspark.ml.feature import VectorAssembler

# Converting lv activepower (kw) variable as label
spark_df = spark_df.withColumn("label", spark_df["lv activepower (kw)"])

# Defining the variables to be used
variables = ["month", "hour", "wind speed (m/s)", "wind direction (°)"]
vector_assembler = VectorAssembler(inputCols=variables, outputCol="features")
va_df = vector_assembler.transform(spark_df)

# Combining features and label column
final_df = va_df.select("features", "label")
final_df.show(10, truncate=False)

## Train Test Split

**Now we can split our dataset into train and test datasets.**

In [ ]:
splits = final_df.randomSplit([0.8, 0.2])
train_df = splits[0]
test_df = splits[1]

print("Train dataset: ", train_df.count())
print("Test dataset : ", test_df.count())

## Creating the Initial Model

**I am going to use Random Forest Regressor for this study.**

In [ ]:
from pyspark.ml.regression import RandomForestRegressor

rf = RandomForestRegressor(featuresCol="features", labelCol="label")

# Training the model with train data
rf_model = rf.fit(train_df)

# Predicting using the test data
y_pred = rf_model.transform(test_df)

# Initial look at the target and predicted values
y_pred.select("label", "prediction").show(20)

**Let's evaluate our model's success.**

In [ ]:
# Initial model success
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="label")

print("R2 SCORE : ", evaluator.evaluate(y_pred, {evaluator.metricName: "r2"}))
print("MAE      : ", evaluator.evaluate(y_pred, {evaluator.metricName: "mae"}))
print("RMSE     : ", evaluator.evaluate(y_pred, {evaluator.metricName: "rmse"}))

**R2 score means, real power production's 97% variability can be explained by the ML model.**

**MAE is the mean absolute difference between the real and predicted power production.**

**RMSE is the square root of mean squared difference between the real and predicted values.**

**Even though the R2 is high, we should also check the MAE and RMSE values with the real value's summary statistics.**

**One can tune the hyperparameters to increase the model success. But I this look good enough for me.**

## Comparing Real, Theoritical and Predicted Power Productions

**I am going to use sample_df for comparing the actual, theoritical and the model power productions.**

**I am going to use sample_df for comparing the actual, theoritical and the model power productions.**

In [ ]:
# Converting sample_df back to Spark dataframe
eva_df = spark.createDataFrame(sample_df)

# Converting lv activepower (kw) variable as label
eva_df = eva_df.withColumn("label", eva_df["lv activepower (kw)"])

# Defining the variables to be used
variables = ["month", "hour", "wind speed (m/s)", "wind direction (°)"]
vector_assembler = VectorAssembler(inputCols=variables, outputCol="features")
vec_df = vector_assembler.transform(eva_df)

# Combining features and label column
vec_df = vec_df.select("features", "label")

# Using ML model to predict
preds = rf_model.transform(vec_df)
preds_df = preds.select("label", "prediction").toPandas()

# Compining dataframes to compare
frames = [sample_df[["wind speed (m/s)", "theoretical_power_curve (kwh)"]], preds_df]
sample_data = pd.concat(frames, axis=1)

# Visualizing real, theoritical and predicted power production
plt.figure(figsize=(10, 7))
sns.scatterplot(x="wind speed (m/s)", y="label", alpha=0.5, label="Real Power", data=sample_data)
sns.scatterplot(x="wind speed (m/s)", y="prediction", alpha=0.7, label="Predicted Power", marker="o", data=sample_data)
sns.lineplot(
    x="wind speed (m/s)", y="theoretical_power_curve (kwh)", label="Theoritical Power", color="purple", data=sample_data
)
plt.title("Wind Turbine Power Production Prediction")
plt.ylabel("Power Production (kw)")
plt.legend();

**From the graph above, the model fits better to the real power productions, than the theoritical power production curve.**

## Exporting RF Regressor Model as an ONNX Model

In [ ]:
from onnx.defs import onnx_opset_version
from onnxconverter_common.onnx_ex import DEFAULT_OPSET_NUMBER
from onnxmltools import convert_sparkml
from onnxmltools.convert.common.data_types import FloatTensorType

In [ ]:
TARGET_OPSET = min(DEFAULT_OPSET_NUMBER, onnx_opset_version())

# Define the input types - 'features' is the input column, len(variables) is the number of features (4 in your case)
variables = ["month", "hour", "wind speed (m/s)", "wind direction (°)"]
initial_types = [("features", FloatTensorType([1, len(variables)]))]

onnx_model = convert_sparkml(
    rf_model, "Random Forest Regressor Model", initial_types, spark_session=spark, target_opset=TARGET_OPSET
)

**Looking at the onnx model structure.**

In [ ]:
graph = onnx_model.graph

# Inspect inputs
print("Inputs:")
for inp in graph.input:
    print(f"Name: {inp.name}, Shape: {inp.type.tensor_type.shape}, Type: {inp.type.tensor_type.elem_type}")

# Inspect outputs
print("\nOutputs:")
for out in graph.output:
    print(f"Name: {out.name}, Shape: {out.type.tensor_type.shape}, Type: {out.type.tensor_type.elem_type}")

**Exporting the onnx model.**

In [ ]:
# Save the ONNX model to a file
with Path("rf_model.onnx").open("wb") as f:
    f.write(onnx_model.SerializeToString())

**Testing the onnx model with some input values.**

In [ ]:
import onnxruntime as rt

# Load the ONNX model
sess = rt.InferenceSession("rf_model.onnx", providers=["CPUExecutionProvider"])

# Prepare input as a numpy array (for example)
input_data = np.array(
    [[1.0, 0.0, 5.31133604049682, 259.994903564453]], dtype=np.float32
)  # month, hour, wind speed, wind direction

# Run the model
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name
predictions = sess.run([label_name], {input_name: input_data})

print("Predicted value:", predictions)

**Now we can create an FMU model using MLFMU package.**